Классификация, которую я использовала для размеки:
1. Гендерно-маркированная номинация лица
2. Гендерно-нейтральная номинация лица
3. Дружеское или фамильярное обращение
4. родственник
5. Лицо, состоящее в дружеских отношениях с говорящим
6. Лицо, состоящее в романтических отношениях с говорящим
7. Молодой человек
8. Взрослый человек
9. Пожилой человек
10. Лицо младше или одного возраста с говорящим
0. Воплощение свойств и качеств, присущих данному полу


# Импорты

In [ ]:
!pip install evaluate

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
import spacy
import torch

from datasets import  Dataset, load_metric
from evaluate import load
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from torch import nn
from torch.utils.data import WeightedRandomSampler
from transformers import  AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Работа с данными

In [ ]:
df = pd.read_excel('/content/Совсем окончательный датасет.xlsx')
df.head()

In [ ]:
df.info()

In [ ]:
class_counts = df['1 level'].value_counts()
class_counts

In [ ]:
df['conn'].value_counts()

In [ ]:
copy_df = df.copy()

In [ ]:
#перекодирую коннотацию из текста в числа
names={'conn': {'negative': 0, 'neutral': 1, 'positive': 2}}
coded_result = copy_df.replace(names)
coded_result

Номера  классов второго уровння начинаются не с 0, поэтому перекодирую их в номера с 0 по 10. Получилось, что классы с первого по третий сохраняют свое обозначение, а остальные меняются в соответствии со словарем ниже

Пока сделаю самый простой вариант и попробую обучить на классах второго уровня, поэтому удаляю ненужные колонки

In [ ]:
mixed_df = coded_result.drop(['Full context', 'conn'], axis=1)

In [ ]:
mixed_df

In [ ]:
def tokenize(data, tokenizer):
    return tokenizer(
        data["text"],
        padding="max_length",
        truncation=True,
        max_length=120
    )

In [ ]:
def compute_metrics(eval_pred):
    accuracy_metric = load("accuracy")
    precision_metric = load("precision")
    recall_metric = load("recall")
    f1_metric = load("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    calculated_metric = {}
    #отдельно вычисляю accuracy, т.к. для нее average не предусмотрен
    calculated_metric.update(accuracy_metric.compute(predictions=predictions,
                                                     references=labels))

    for metric in [precision_metric, recall_metric, f1_metric]:
      calculated_metric.update(metric.compute(predictions=predictions,
                                              references=labels,
                                              average="weighted"))  #при дисбалансе классов
    #метрики для каждого класса
    unique_labels = np.unique(labels)
    if len(unique_labels) > 2:
        for label in unique_labels:
             calculated_metric.update({
                f"precision_class_{label}": precision_score(
                    labels, predictions, average=None)[label],
                f"recall_class_{label}": recall_score(
                    labels, predictions, average=None)[label],
                f"f1_class_{label}": f1_score(
                    labels, predictions, average=None)[label]
            })
    return calculated_metric


# Предсказание значений

In [ ]:
num_labels =  len(set(mixed_df["1 level"]))
num_labels

In [ ]:
model_name = "DeepPavlov/rubert-base-cased-conversational"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    problem_type="single_label_classification")

Так как датасет у меня небольшой, в качестве обучающей выборки возьму 0.7 от всего датасета, а на  валидационную и тестовую уйдет по 0.15 датасета

In [ ]:
def split_data():
  X_train, X,  y_train, y = train_test_split(mixed_df['marked_text'],
                                                  mixed_df['1 level'],   #столбец, значения в котором хотим  предсказать
                                                  test_size = 0.3,  #доля тестовой выборки от общего числа данных
                                                  random_state = 42,
                                                  stratify = mixed_df['1 level'])  #столбец,  по которому стратифицируем данные

  X_test, X_val, y_test, y_val = train_test_split(X, y,
                                                test_size = 0.5,
                                                random_state = 42,
                                                stratify=y)
  return (
        Dataset.from_dict({"text": X_train.tolist(), "labels": y_train.tolist()}),
        Dataset.from_dict({"text": X_val.tolist(), "labels": y_val.tolist()}),
        Dataset.from_dict({"text": X_test.tolist(), "labels": y_test.tolist()})
    )

Ниже два мейна, которые я запускала с разными гиперпараметрами. Рабочий - первый, результатов второго я не дождалась, так как файнтюнинг, похоже, длился бы 3 часа или больше

In [ ]:
def main():
  X_train, X_val, X_test = split_data()
  tokenized_train = X_train.map(
        lambda x: tokenize(x, tokenizer),
        batched=True)

  tokenized_val = X_val.map(
        lambda x: tokenize(x, tokenizer),
        batched=True)


  tokenized_test = X_test.map(
        lambda x: tokenize(x, tokenizer),
        batched=True)

  training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=32,
    learning_rate=5e-5,
    num_train_epochs=3,
    save_strategy="no",
    logging_steps=50)

  trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        compute_metrics=compute_metrics,)

  print("происходит файнтюнинг")
  trainer.train()

  print("результаты на валидационной выборке")
  val_results = trainer.evaluate()
  for key, value in val_results.items():
      if key.startswith('eval_'):
          print(f"{key[5:]}: {value:.4f}")



  model.save_pretrained("./fine_tuned_model")
  tokenizer.save_pretrained("./fine_tuned_model")
  print("Модель сохранена в fine_tuned_model/")


if __name__ == "__main__":
    main()

Судя по метрикам, нулевой класс (бывший класс №8) не предсказывается совсем. Это предсказуемо, учитывая, что его экземпляров  не очень много, всего 60. Класс 6 и 8 тоже предсказывается не очень хорошо, но их экземпляров тоже не очень много, около 70-80

In [ ]:
#сохраняю в гугл драйв папку с моделью
!cp -r /content/fine_tuned_model /content/drive/MyDrive/2level_model

In [ ]:
finetuned_model_path = "/content/drive/MyDrive/2level_model"
finetuned_tokenizer_path = "/content/drive/MyDrive/2level_model"
ft_model = AutoModelForSequenceClassification.from_pretrained(finetuned_model_path)
ft_tokenizer = AutoTokenizer.from_pretrained(finetuned_tokenizer_path)

In [ ]:
#создание confusion matrix
def plot_confusion_matrix_11_classes(test_dataset):
    trainer = Trainer(model=ft_model)
    predictions = trainer.predict(test_dataset)
    preds = np.argmax(predictions.predictions, axis=1)
    labels = test_dataset["labels"]

    #создаем подписи для классов
    class_names = [f"Class {i}" for i in range(11)]

    #Confusion Matrix
    cm = confusion_matrix(labels, preds)

    #перевожу в проценты
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    #настройка размера графика для 11 классов
    plt.figure(figsize=(15, 12))

    #визуализация
    sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap="Blues",
                xticklabels=class_names,
                yticklabels=class_names)
    plt.title("Normalized Confusion Matrix (11 классов)", pad=20, fontsize=16)
    plt.xlabel("Predicted", fontsize=14)
    plt.ylabel("True", fontsize=14)

    #сохранение
    plt.savefig("/content/drive/MyDrive/2level_model/confusion_matrix_11_classes.png",
                bbox_inches='tight', dpi=300)
    plt.show()

    #Classification Report
    print(classification_report(labels, preds, target_names=class_names, digits=4))

In [ ]:
X_train, X_val, X_test = split_data()

tokenized_test = X_test.map(
        lambda x: tokenize(x, tokenizer),
        batched=True)

In [ ]:
plot_confusion_matrix_11_classes(tokenized_test)

## Проверяю работу модели на отдельных  примерах

In [ ]:
def predict_meaning (context: str, lexeme: str, model, tokenizer):
  marked_context = context
  inputs = tokenizer(
        marked_context,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    )

  with torch.no_grad():
      outputs = model(**inputs)

  probs = torch.softmax(outputs.logits, dim=-1).cpu().numpy()[0]
  predicted_class = np.argmax(probs)

  result = {"lexeme": lexeme,
            "context": marked_context,
            "predicted_class": int(predicted_class),
            "confidence": float(probs[predicted_class]),
            "all_probs": probs.tolist()
        }
  return result

In [ ]:
res1 = predict_meaning("мы с моей [TGT]чувихой[/TGT] вчера были на концерте", "чувиха", ft_model, ft_tokenizer)
res1

In [ ]:
res = predict_meaning("я всегду буду твоим [TGT]бро[/TGT], даже если мы поссоримся", "бро", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("я уже все понял по каждому [TGT]бро[/TGT] и можешь не пытаться меня переубедить", "бро", ft_model, ft_tokenizer)
res


In [ ]:
res = predict_meaning("будь [TGT]бро[/TGT], помоги нам выиграть в этом конкурсе", "бро", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("спасибо [TGT]бро[/TGT] за помощь в организации вечеринки", "бро", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("ты мой [TGT]бро[/TGT] навсегда, только скажи мне, поедешь ли ты с нами в москву", "бро", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("Затем он говорил [TGT]мальчикам[/TGT]: «Именно так работает поэзия.", "мальчик", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("Какой ты гениальный [TGT]мальчик[/TGT]! Я бы ведь сама не догадалась так сделать", "мальчик", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("Хочу оставаться [TGT]женщиной[/TGT], а не превращаться в  сапожника", "женщина", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("мы с [TGT]пацанами[/TGT] часто собираемся вместе.", "пацан", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("мы с моими [TGT]девами[/TGT] пошли вчера вместе за кофе.", "дева", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("у [TGT]деда[/TGT] дома или у меня решили собраться.", "дед", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("[TGT]дед[/TGT] мне представлялся человеком очень спокойным", "дед", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("зачем вы так строго с ним, жалко [TGT]деда[/TGT]", "дед", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("тебе [TGT]деды[/TGT] местные ничего не сказали?", "дед", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("[TGT]мужчина[/TGT] услышал крики и решил помочь несчастным", "мужчина", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("ради такой [TGT]женщины[/TGT] я был бы готов на многое", "женщина", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("[TGT]женщина[/TGT] показалась нам очень понимающей", "женщина", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("вокруг так много красивых [TGT]женщин[/TGT].", "женщина", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("[TGT]женщина[/TGT] сказала, что пойдет с нами.", "женщина", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("меня очень тронули слова этого [TGT]деда[/TGT]", "дед", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("зачем [TGT]герл[/TGT] обижаешь", "герл", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("[TGT]пацаны[/TGT] это вам во дворе сказали?", "пацан", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("такая хот [TGT]герл[/TGT] мне навстречу вышла", "герл", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("бар [TGT]герл[/TGT] сегодня не та, что обычно", "герл", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("надоела эта клин [TGT]герл[/TGT]", "герл", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("он такой эстетик [TGT]мэн[/TGT]", "мэн", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("ему сказал какой-то [TGT]челик[/TGT] давай вместе съездим", "чел", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("тебе же [TGT]чел[/TGT] из германии кроссы привез", "чел", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("[TGT]чел[/TGT], покупающий плойку в первый раз может и не разберется, ", "чел", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("[TGT]чел[/TGT] с какой то сумкой это странно", "чел", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("[TGT]чел[/TGT] не до конца понял этот фильм", "челик", ft_model, ft_tokenizer)
res

In [ ]:
res = predict_meaning("[TGT]челик[/TGT] просто все еще бомбит что не получилось", "челик", ft_model, ft_tokenizer)
res

# Предсказание коннотации

In [ ]:
conn_df = coded_result.drop(['Full context', '1 level'], axis=1)

In [ ]:
conn_df

In [ ]:
num_labels_conn =  len(set(conn_df["conn"]))
num_labels_conn

In [ ]:
def split_data_conn():
  X_train, X, y_train, y = train_test_split(conn_df['marked_text'],
                                                  conn_df['conn'],   #столбец, значения в котором хотим  предсказать
                                                  test_size = 0.3,  #доля тестовой выборки от общего числа данных
                                                  random_state = 42,
                                                  stratify = conn_df['conn'])  #столбец,  по которому стратифицируем данные

  X_test, X_val, y_test, y_val = train_test_split(X, y,
                                                test_size = 0.5,
                                                random_state = 42,
                                                stratify=y)
  return (
        Dataset.from_dict({"text": X_train.tolist(), "labels": y_train.tolist()}),
        Dataset.from_dict({"text": X_val.tolist(), "labels": y_val.tolist()}),
        Dataset.from_dict({"text": X_test.tolist(), "labels": y_test.tolist()})
    )

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class CustomTrainer(Trainer):
    def __init__(self, *args, sampler=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.sampler = sampler

    def get_train_dataloader(self):
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")

        if self.sampler is not None:
            return torch.utils.data.DataLoader(
                self.train_dataset,
                batch_size=self.args.train_batch_size,
                sampler=self.sampler,
                collate_fn=self.data_collator,
                drop_last=self.args.dataloader_drop_last,
                num_workers=self.args.dataloader_num_workers,
                pin_memory=self.args.dataloader_pin_memory,
            )
        else:
            return super().get_train_dataloader()

In [ ]:
def main():
  X_train, X_val, X_test = split_data_conn()
  labels = np.array(X_train["labels"])

  class_weights = compute_class_weight(
        class_weight="balanced",
        classes=np.unique(labels),
        y=labels)

  class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

  model_name = "DeepPavlov/rubert-base-cased-conversational"
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model_conn = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                                  num_labels=3,
    problem_type="single_label_classification")

  model_conn.classifier = nn.Linear(model_conn.config.hidden_size, 3)
  model_conn.loss_fct = nn.CrossEntropyLoss(weight=class_weights)

  sampler = WeightedRandomSampler(weights=class_weights[labels],
                                  num_samples=len(labels),
                                  replacement=True)


  tokenized_train = X_train.map(
        lambda x: tokenize(x, tokenizer),
        batched=True)


  tokenized_val = X_val.map(
        lambda x: tokenize(x, tokenizer),
        batched=True)


  tokenized_test = X_test.map(
        lambda x: tokenize(x, tokenizer),
        batched=True)

  training_args = TrainingArguments(
    output_dir="./balanced_results",
    per_device_train_batch_size=32,
    learning_rate=2e-5,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="no",
    logging_steps=50)

  trainer = CustomTrainer(
        model=model_conn,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        compute_metrics=compute_metrics,
        sampler=sampler)

  print("происходит файнтюнинг")
  trainer.train()

  print("результаты на валидационной выборке")
  val_results = trainer.evaluate()
  for key, value in val_results.items():
      if key.startswith('eval_'):
          print(f"{key[5:]}: {value:.4f}")


  model_conn.save_pretrained("./fine_tuned_conn_model_sampling")
  tokenizer.save_pretrained("./fine_tuned_conn_model_sampling")
  print("Модель сохранена в fine_tuned_conn_model_sampling/")


if __name__ == "__main__":
    main()

In [ ]:
!cp -r /content/fine_tuned_conn_model_sampling /content/drive/MyDrive/conn_sampling__model

In [ ]:
ft_conn_model_path = "/content/drive/MyDrive/conn_sampling__model"
ft_conn_tokenizer_path = "/content/drive/MyDrive/conn_sampling__model"
ft_conn_model = AutoModelForSequenceClassification.from_pretrained(ft_conn_model_path)
ft_conn_tokenizer = AutoTokenizer.from_pretrained(ft_conn_tokenizer_path)

In [ ]:
def plot_confusion_matrix(test_dataset):
    trainer = Trainer(model=ft_conn_model)
    predictions = trainer.predict(test_dataset)
    preds = np.argmax(predictions.predictions, axis=1)
    labels = test_dataset["labels"]

    cm = confusion_matrix(labels, preds)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap="Blues",
                xticklabels=["Class 0", "Class 1", "Class 2"],
                yticklabels=["Class 0", "Class 1", "Class 2"])
    plt.title("Normalized Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.savefig("/content/drive/MyDrive/conn_sampling__model/confusion_matrix.png")
    plt.show()

    print(classification_report(labels, preds, target_names=["Class 0", "Class 1", "Class 2"]))

In [ ]:
X_train, X_val, X_test = split_data_conn()

tokenized_test = X_test.map(
        lambda x: tokenize(x, ft_conn_tokenizer),
        batched=True)

In [ ]:
plot_confusion_matrix(tokenized_test)

## Проверяю работу модели на отдельных примерах

In [ ]:
conn = predict_meaning("Какой ты гениальный мальчик, я бы ведь мама не догадалась так сделать!", "мальчик", ft_conn_model, ft_conn_tokenizer)
conn

In [ ]:
conn = predict_meaning("ну что ты как девочка? Реши хоть что-то сам и не ной!", "девочка", ft_conn_model, ft_conn_tokenizer)
conn

In [ ]:
conn = predict_meaning("моя герл - огонь, с ней не соскучишься!", "герл", ft_conn_model, ft_conn_tokenizer)
conn

In [ ]:
conn = predict_meaning("Чел реально заморочился", "чел", ft_conn_model, ft_conn_tokenizer)
conn

# Бейзлайн

In [ ]:
X_train, X_val, X_test, y_train, y_test = split_data()

In [ ]:
class_counts = np.bincount(y_train)
class_probabilities = class_counts / len(y_train)
y_pred_baseline = np.random.choice(len(class_probabilities), size=len(y_test), p=class_probabilities)
baseline_precision = precision_score(y_test, y_pred_baseline, average="weighted")
baseline_f1 = f1_score(y_test, y_pred_baseline, average="weighted")
baseline_recall = recall_score(y_test, y_pred_baseline, average="weighted")

print(f"Precision бейзлайна: {baseline_precision}: Recall {baseline_recall}; f1: {baseline_f1}")

In [ ]:
X_train, X_val, X_test, y_train, y_test = split_data_conn()

In [ ]:
class_counts = np.bincount(y_train)
class_probabilities = class_counts / len(y_train)
y_pred_baseline = np.random.choice(len(class_probabilities), size=len(y_test), p=class_probabilities)
baseline_precision = precision_score(y_test, y_pred_baseline, average="weighted")
baseline_f1 = f1_score(y_test, y_pred_baseline, average="weighted")
baseline_recall = recall_score(y_test, y_pred_baseline, average="weighted")

print(f"Precision бейзлайна: {baseline_precision}: Recall {baseline_recall}; f1: {baseline_f1}")